# Loading Crypto Data

In [3]:
import numpy as np

import os

import pickle

import yfinance as yf

In [4]:
save_path = '../data/crypto/'
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [5]:
tickers = ['BTC-USD', 'ETH-USD', 'USDT-USD']#, 'BNB-USD', 'SOL-USD']
names = ['Bitcoin', 'Ethereum', 'Tether']#, 'BNB', 'Solana']

In [6]:
data = yf.download(  # or pdr.get_data_yahoo(...
            # tickers list or string as well
            tickers = tickers,

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            #period = "1mo",

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            #interval = "1d",

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            #group_by = 'Ticker', #Ticker

            # adjust all OHLC automatically
            # (optional, default is False)
            #auto_adjust = False,

            # download pre/post regular market hours data
            # (optional, default is False)
            #prepost = False,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            #threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            #proxy = None
        )

[*********************100%%**********************]  3 of 3 completed


In [7]:
vix_data = yf.download(  # or pdr.get_data_yahoo(...
            # tickers list or string as well
            tickers = ['^VIX'],

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            #period = "1mo",

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            #interval = "1d",

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            #group_by = 'Ticker', #Ticker

            # adjust all OHLC automatically
            # (optional, default is False)
            #auto_adjust = False,

            # download pre/post regular market hours data
            # (optional, default is False)
            #prepost = False,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            #threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            #proxy = None
        )

[*********************100%%**********************]  1 of 1 completed


In [8]:
vix = vix_data['Adj Close']

In [9]:
ln_vix = np.log(vix).diff()

In [10]:
ln_vix

Date
1990-01-02         NaN
1990-01-03    0.053640
1990-01-04    0.055079
1990-01-05    0.045266
1990-01-08    0.007431
                ...   
2024-05-13    0.080349
2024-05-14   -0.013324
2024-05-15   -0.075026
2024-05-16   -0.002413
2024-05-17   -0.035235
Name: Adj Close, Length: 8661, dtype: float64

In [11]:
adj_close = data['Adj Close']

In [12]:
adj_close = adj_close.dropna(inplace=False)

In [13]:
ln_diff = np.log(adj_close).diff()

In [14]:
ln_diff

,BTC-USD,ETH-USD,USDT-USD
Date,,,
2017-11-09,NaN,NaN,NaN
2017-11-10,-0.076400,-0.069790,-0.002155
2017-11-11,-0.040163,0.050270,0.002958
2017-11-12,-0.066248,-0.021758,0.003443
2017-11-13,0.097510,0.028204,-0.003086
...,...,...,...
2024-05-14,-0.021674,-0.023396,-0.000264
2024-05-15,0.073804,0.052697,0.000955
2024-05-16,-0.015756,-0.030736,-0.000449


In [17]:
import pandas as pd
result = pd.concat([ln_vix, ln_diff], axis=1)

In [21]:
result.dropna(axis=0, inplace=True)

In [33]:
bitcoin_data = result[tickers]
vix_data = result['Adj Close']

In [34]:
cov_val = (100*bitcoin_data).rolling(21).cov().unstack()

In [35]:
result = pd.concat([vix_data, cov_val], axis=1)

In [37]:
result.dropna(axis=0, inplace=True)

In [38]:
result

,Adj Close,"(BTC-USD, BTC-USD)","(BTC-USD, ETH-USD)","(BTC-USD, USDT-USD)","(ETH-USD, BTC-USD)","(ETH-USD, ETH-USD)","(ETH-USD, USDT-USD)","(USDT-USD, BTC-USD)","(USDT-USD, ETH-USD)","(USDT-USD, USDT-USD)"
Date,,,,,,,,,,
2017-12-11,-0.025371,52.756427,2.303034,3.617404,2.303034,38.673043,1.047354,3.617404,1.047354,0.657130
2017-12-12,0.060247,45.646642,-5.019633,2.933244,-5.019633,56.548861,6.625183,2.933244,6.625183,2.144126
2017-12-13,0.025872,49.368990,-7.144693,5.658065,-7.144693,57.495575,5.512626,5.658065,5.512626,3.402179
2017-12-14,0.029997,49.428803,-6.091818,5.899000,-6.091818,57.830450,5.968775,5.899000,5.968775,3.585494
2017-12-15,-0.107587,48.072235,-5.491736,6.129208,-5.491736,57.995796,5.712614,6.129208,5.712614,3.601714
...,...,...,...,...,...,...,...,...,...,...
2024-05-13,0.080349,8.980531,6.500948,0.104245,6.500948,5.772243,0.069505,0.104245,0.069505,0.002339
2024-05-14,-0.013324,8.622980,6.481151,0.095308,6.481151,5.846570,0.068022,0.095308,0.068022,0.002128
2024-05-15,-0.075026,11.429830,8.698318,0.130201,8.698318,7.585116,0.095314,0.130201,0.095314,0.002556


In [39]:
data_dict = {'data': result,
             'tickers': tickers,
             'names': names
            }

In [40]:
with open(''.join((save_path, 'crypto.pkl')), 'wb') as f:
    pickle.dump(data_dict, f)